[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/blhuillier/STAR_2025_GP/blob/main/Notebooks/GP_solution.ipynb
)

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
class Kernel():
  def __init__(self,sigma_f,lf):
    self.lf = lf
    self.sigma_f = sigma_f

  def k(self,xx,yy):
    """x: vector
    y: vector
    """
    print (xx.shape,yy.shape)
    return self.sigma_f**2 * np.exp(-.5*(xx-yy)**2/self.lf**2)




In [ ]:
class GP():
  def __init__(self,x,y,sigma,kernel):
    self.x = x
    self.y = y
    self.sigma = sigma
    self.kernel = kernel
    return

  def do_regression(self,xstar,kernel,mprime=0,mprimestar=0):
    '''Perform the actual GP regression
    input:
    xstar: training points
    kernel: Kernel isntance
    mprime: prior mean function (m(x))
    mprimestar: prior mean function m*(x*)
    '''

    # this has sahpe (n,n)
    KK = self.kernel.k(
        XX[np.newaxis,:],
        XX[:,np.newaxis]
      )


    # This has shape (n*,n)
    Kstar = self.kernel.k(
        Xstar[:,np.newaxis],
        XX[np.newaxis,:]
    )

    # This has shape (n*,n*)
    Kstarstar = self.kernel.k(
        Xstar[:,np.newaxis],
        Xstar[np.newaxis,:],
    )
    # This is K+sigma^2 I:
    KC = KK + sigma**2*np.eye(NN)
    LL = np.linalg.cholesky(KC)
    # print(LL)
    alpha = np.linalg.solve(
        LL.T,
        np.linalg.solve(LL,YY)
    )


    self.fstar = Kstar@alpha# vv = np.linalg.solve (Kstar,LL)

    vv = np.array([np.linalg.solve(LL,Kstar[ii,:]) for ii in range(nstar)])
    print (vv.shape)
    self.Cstar = Kstarstar - vv@vv.T
    self.sigmastar = np.sqrt(np.diag(self.Cstar))

    LML = -.5*YY.T@alpha - np.linalg.slogdet(LL)[1] - NN/2*np.log(2*np.pi)
    print (LML)

    # sigmaf: 2, lf: 2: LML: -68.24408191515852
    # lf = 0.5: LML = -93.14685243016704



In [ ]:
# generate data

aa = 1
bb = 1
kk = 1

def func(xx,aa=1,bb=1,kk=1):
  return aa*xx + bb + np.sin(kk*xx)

NN = 30
seed = 0
r = np.random.RandomState(seed)

XX = r.uniform(0,10,size=NN)
print (XX)

# Noise
sigma = 1
noise = sigma*r.randn(NN)
# data
YY = func(XX)+noise
# Test points:
nstar = 201
Xstar = np.linspace(0,10,num=nstar)


In [ ]:
# Data Covariance
sigma_f = 2
lf = 2
K = Kernel(sigma_f,lf)

In [ ]:

gp = GP(x=XX,y=YY,sigma=sigma,kernel=K)
gp.do_regression(xstar=Xstar,kernel=K)
fstar = gp.fstar
Cstar = gp.Cstar
sigmastar = np.sqrt(np.diag(Cstar))

In [ ]:

fig,ax = plt.subplots(1)
ax.errorbar(XX,YY,yerr=sigma,ls='',marker='.')
ax.plot(Xstar,func(Xstar),ls=':',label='Truth')
ax.plot(Xstar,fstar,label='$f_*$')
ax.fill_between(Xstar,fstar+sigmastar,fstar-sigmastar,alpha=.3)
ax.legend()